3.3. CIFAR-10 画像認識

CIFAR-10 は、カナダのトロント大学が提供する、10種類の画像を分類するためのデータセットです。このデータセットは、32x32ピクセルのカラー画像 60,000枚で構成されています。そのうち、50,000枚が訓練用、10,000枚がテスト用になっています。画像は、飛行機、自動車、鳥、猫、鹿、犬、カエル、馬、船、トラックの10種類です。

In [28]:
import os
import keras

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import Flatten, Dense, Dropout

from keras.datasets import cifar10

from keras.optimizers import RMSprop

from keras.callbacks import TensorBoard, ModelCheckpoint


In [41]:
# LeNet configuration

def network(input_shape, num_classes):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  return model

In [42]:
class CIFAR10Dataset():
  """
  CIFAR-10 dataset.
  attributes:
    - image_shape: image shape
    - num_classes: number of classes
  """
  def __init__(self):
    self.image_shape = (32, 32, 3)
    self.num_classes = 10

  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    x_train /= 255
    x_test /= 255

    y_train = keras.utils.to_categorical(y_train, self.num_classes)
    y_test = keras.utils.to_categorical(y_test, self.num_classes)

    return x_train, y_train, x_test, y_test
  
  def preprocess(slef, data, label_data=False):
    if label_data:
      data = keras.utils.to_categorical(data, self.num_classes)
    else:
      data = data.astype('float32')
      data /= 255
      shape = (data.shape[0],) + self.image_shape
      return data

In [43]:
class Trainer():
  def __init__(self, model, loss, optimizer):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    self.verbose = 1
    self.log_dir = os.path.join(os.getcwd(), 'logdir')
    
    
    self.model_file_name = 'model_file.hdf5'

  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)

    self._target.fit(
      x_train, y_train,
      batch_size=batch_size, epochs=epochs,
      validation_split=validation_split,
      callbacks=[
        TensorBoard(log_dir=self.log_dir),
        ModelCheckpoint(os.path.join(self.log_dir, self.model_file_name), save_best_only=True)
      ],
      verbose=self.verbose)

In [44]:
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

model = network(dataset.image_shape, dataset.num_classes)
trainer = Trainer(model, loss='categorical_crossentropy', optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

# show result
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
313/313 [==============================] - 8s 22ms/step - loss: 1.7259 - accuracy: 0.3789 - val_loss: 1.5396 - val_accuracy: 0.4675
Epoch 2/12
313/313 [==============================] - 7s 21ms/step - loss: 1.3734 - accuracy: 0.5126 - val_loss: 1.2589 - val_accuracy: 0.5535
Epoch 3/12
313/313 [==============================] - 7s 21ms/step - loss: 1.2463 - accuracy: 0.5621 - val_loss: 1.1621 - val_accuracy: 0.5909
Epoch 4/12
313/313 [==============================] - 7s 22ms/step - loss: 1.1869 - accuracy: 0.5865 - val_loss: 1.0662 - val_accuracy: 0.6301
Epoch 5/12
313/313 [==============================] - 7s 23ms/step - loss: 1.1588 - accuracy: 0.5999 - val_loss: 1.1204 - val_accuracy: 0.6142
Epoch 6/12
313/313 [==============================] - 7s 21ms/step - loss: 1.1815 - accuracy: 0.6023 - val_loss: 1.1226 - val_accuracy: 0.6176
Epoch 7/12
313/313 [==============================] - 7s 22ms/step - loss: 1.2406 - accuracy: 0.5937 - val_loss: 1.1283 - val_accuracy: 0.6170

参考書と同じコードだと思うが、結果が大違いだ！

In [45]:
# check tensorboard
# !tensorboard --logdir=logdir

**ネットワークをもっと深くすることによる改善**

In [46]:
# LeNet configuration

def network(input_shape, num_classes):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  return model

In [47]:
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

model = network(dataset.image_shape, dataset.num_classes)
trainer = Trainer(model, loss='categorical_crossentropy', optimizer=RMSprop())
trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

# show result
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
313/313 [==============================] - 10s 28ms/step - loss: 1.8976 - accuracy: 0.3102 - val_loss: 1.5836 - val_accuracy: 0.4160
Epoch 2/12
313/313 [==============================] - 8s 26ms/step - loss: 1.5087 - accuracy: 0.4529 - val_loss: 1.3280 - val_accuracy: 0.5209
Epoch 3/12
313/313 [==============================] - 8s 25ms/step - loss: 1.3483 - accuracy: 0.5199 - val_loss: 1.2712 - val_accuracy: 0.5362
Epoch 4/12
313/313 [==============================] - 8s 25ms/step - loss: 1.3157 - accuracy: 0.5497 - val_loss: 1.1755 - val_accuracy: 0.5949
Epoch 5/12
313/313 [==============================] - 8s 25ms/step - loss: 1.5503 - accuracy: 0.5425 - val_loss: 1.6928 - val_accuracy: 0.5319
Epoch 6/12
313/313 [==============================] - 8s 25ms/step - loss: 2.7145 - accuracy: 0.4997 - val_loss: 2.9195 - val_accuracy: 0.5418
Epoch 7/12
313/313 [==============================] - 8s 25ms/step - loss: 6.6445 - accuracy: 0.4588 - val_loss: 7.5319 - val_accuracy: 0.462